In [1]:
from imputation import core_utils, core_imputation_model
import numpy as np
from tqdm.notebook import tqdm

# Loading the Data

`core_utils.get_data_panel` loads the data from the corresponding `data_path`, which is a feather file shared on Google drive. We have to use Gdrive as it is too large to host on Github. The data file it contains the characteristic percentile ranks as a numpy array of shape TxNxL where T is the number of dates, N the number of stocks, and L the number of characteristics. The file also includes the raw characteristics, the characteristic namess, the dates and permnos

In [2]:
data_path = "data/example_data.fthr"
percentile_rank_chars, chars, date_vals, permnos = core_utils.get_data_panel(
    path=data_path, computstat_data_present_filter=True,start_date=19770000)

In [3]:
char_groupings = core_utils.CHAR_GROUPINGS

# Running Imputations

In this section we will run the imputation method described in the paper.

Two methods we want to highlight are
- `core_imputation_model_new.run_imputation`
- `core_imputation_model_new.fit_factors_and_loadings`

The first code runs the full method as described in the paper, including potentially different time series information sets depending on the arguments given.

The second code estimates the XS factor model. 

The examples below correspond to global and local fits. The parameters are documented in the function definition. 

In [4]:
T, N, L = percentile_rank_chars.shape

## Estimating the Model

We start with the local estimation. In this case, we show how to estimate either the purely cross-sectional model (local XS) or the cross-sectional model with backwards time series information (local B-XS). 

We would like to emphasize two parameters in this estimation. This first is the number of cross-sectional factors K: `n_xs_factors` the second is the cross-sectional factor regularization gamma: `xs_factor_reg`.

These two hyperparameters have a significant impact on the performance of the model, and should be selected carefully. The parameters we use in this example are selected for the data-set from Missing Financial Data, and should not be considered default aprameters for alternative data-sets.

In [5]:
T, N, L = percentile_rank_chars.shape

imputation = core_imputation_model.run_imputation(
    percentile_rank_chars, 
    n_xs_factors=20,
    time_varying_loadings=True,
    xs_factor_reg=0.01 / L,
    use_bw_ts_info=False, 
    include_ts_residuals=True,
    min_xs_obs=1
)

bw_xs_imputation = core_imputation_model.run_imputation(
    percentile_rank_chars, 
    n_xs_factors=20,
    time_varying_loadings=True,
    xs_factor_reg=0.01 / L,
    use_bw_ts_info=True, 
    include_ts_residuals=True,
    min_xs_obs=1
)

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   4 out of  12 | elapsed:    8.7s remaining:   17.4s
[Parallel(n_jobs=30)]: Done   7 out of  12 | elapsed:    8.9s remaining:    6.4s
[Parallel(n_jobs=30)]: Done  10 out of  12 | elapsed:    9.2s remaining:    1.8s
[Parallel(n_jobs=30)]: Done  12 out of  12 | elapsed:    9.6s finished


  0%|          | 0/12 [00:00<?, ?it/s]

resids rmse are  0.09522440538615277


0it [00:00, ?it/s]

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   4 out of  12 | elapsed:    8.6s remaining:   17.2s
[Parallel(n_jobs=30)]: Done   7 out of  12 | elapsed:    8.9s remaining:    6.3s
[Parallel(n_jobs=30)]: Done  10 out of  12 | elapsed:    9.3s remaining:    1.9s
[Parallel(n_jobs=30)]: Done  12 out of  12 | elapsed:   10.4s finished


  0%|          | 0/12 [00:00<?, ?it/s]

resids rmse are  0.09522440538615277


0it [00:00, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

In [6]:
gamma_ts, lmbda = core_imputation_model.fit_factors_and_loadings(
    char_panel=percentile_rank_chars, 
    min_chars=1, 
    K=20, 
    num_months_train=T,
    reg=0.01 / L,
    time_varying_lambdas=True,
    eval_data=None,
    run_in_parallel=True
)

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   4 out of  12 | elapsed:    8.5s remaining:   17.1s
[Parallel(n_jobs=30)]: Done   7 out of  12 | elapsed:    8.9s remaining:    6.4s
[Parallel(n_jobs=30)]: Done  10 out of  12 | elapsed:    9.6s remaining:    1.9s
[Parallel(n_jobs=30)]: Done  12 out of  12 | elapsed:    9.7s finished


  0%|          | 0/12 [00:00<?, ?it/s]

resids rmse are  0.0951762749598058


# On the Selction of the Number of Factors and Regularization

Below we show the plots from Figures 8 \& 9 in the paper. These figures illustrate how to determine the optimal regularization and number of factors. In more detail, we evaluate the out-of-sample performance of the model for different number of factors and regularization across a grid of these choices.

![example_of_cval.png](example_of_cval.png)

![reg_cval.png](reg_cval.png)

The `core_imputation_model_new.fit_factors_and_loadings` method allows the use to pass in and argument `eval_data`. This, if provided, is compared against the imputation and the RMSE is reported. This is simple way that hyperparameter choices could be evaluated with the model.